In [ ]:
import numpy as np
import pandas as pd
import easyocr
from collections import Counter
from PIL import Image, ImageDraw
import cv2
from langdetect import detect

In [ ]:
df = pd.read_csv('./Data/resized_mapillary/dataset_mapillary.csv', index_col=0)
df['img_path'] = [f"./Data/resized_mapillary/{index}.jpg" for index in df.index]
df['text'] =  pd.Series(dtype='object')
df['confidence'] =  pd.Series(dtype='object')
#df['bbox'] =  pd.Series(dtype='object') ### For later implementations, when BBOX will be drawn over the detected text
df['flag'] = pd.Series(0)
df['lang'] = pd.Series(None)


# OCR Text Recognition
1. It's important to note that we're looking at a plethora of European languages, which will employ various modifications to the latin script, cyrillic, etc. 
2. As such, multiple OCR readers can be employed. We can run each reader on an image, extract the confidence and various pieces of text above some threshold, and then input the collected data to our classifier.
3. We could also use a reader with multiple languages in the list. This will remove the ability to handle different languages differently, but that may be better to avoid data leakage?
4. It may be necessary to prune the 'Google' labels from the images.


https://www.jaided.ai/easyocr/

In [ ]:
class ocr_reader():

    def __init__(self, langs, recog_network = "standard"):
        self.langs = langs
        self.reader = easyocr.Reader(langs, detect_network = 'dbnet18', recog_network=recog_network)
    
    def read(self, df):
        min_length = 5
        self.df_results = df.copy()
        
        for index, row in self.df_results.iterrows():
            
            results = self.OCR_extraction(row['img_path'], self.reader)
            
            #bbox_ = []
            text = []
            confidence = []
            for result in results:
                if len(result[1]) >= min_length:
                    self.df_results.at[index, "flag"] = 1
                    print(f"For image {index}, ")
                    print(f"Text is {result[1]} with confidence {result[2]}")
                    #bbox_.append(result[0])
                    text.append(result[1])
                    confidence.append(result[2])
                
            #self.df_results.at[index, "bbox"] = bbox_
            self.df_results.at[index, "text"] = text
            self.df_results.at[index, "confidence"] = confidence

        return self.df_results
    
    def OCR_extraction(self, path, reader):
        return reader.readtext(path) # , paragraph=True)
    

In [ ]:
languages = {
    "be": "be",
    "bg": "bg",
    "cs": "cs",
    "cy": "cy",
    "da": "da",
    "de": "de",
    "en": "en",
    "es": "es",
    "et": "et",
    "fr": "fr",
    "ga": "ga",
    "hr": "hr",
    "hu": "hu",
    "is": "is",
    "it": "it",
    "la": "la",
    "lt": "lt",
    "lv": "lv",
    "mt": "mt",
    "nl": "nl",
    "no": "no",
    "pl": "pl",
    "pt": "pt",
    "ro": "ro",
    "ru": "ru",
    "rs_latin": "rs",
    "rs_cyrillic": "rc",
    "sk": "sk",
    "sl": "sl",
    "sq": "sq",
    "sv": "sv",
    "tr": "tr",
    "uk": "uk"
}


In [ ]:
latin_languages = [
    "cs", "cy", "da", "de", "en", "es", "et", "fr", "ga", "hr", "hu", "is", "it", "la", "lt", "lv",
    "mt", "nl", "no", "pl", "pt", "ro", "rs_latin", "sk", "sl", "sq", "sv", "tr"
]

cyrillic_languages = [
    "ru","rs_cyrillic","be","bg","uk","en"
]

Readers = {
"en": ocr_reader(["en"]),
"latin": ocr_reader(latin_languages),
"cyrillic": ocr_reader(cyrillic_languages, recog_network="cyrillic_g2")
}

In [ ]:
latin_df = Readers["latin"].read(df)
print("On cyrillic")
cyrillic_df = Readers["cyrillic"].read(df)

In [ ]:
# Saving so I don't need to run this 3 more times if I fuck something up
latin_df.to_csv("latin_OCR.csv")
cyrillic_df.to_csv("cyrillic_OCR.csv")


##### Resume here

In [ ]:
import ast
latin_df = pd.read_csv("latin_OCR.csv", index_col=0, converters={'text': ast.literal_eval})
cyrillic_df = pd.read_csv("cyrillic_OCR.csv", index_col=0, converters={'text': ast.literal_eval})
latin_df['confidence'] = latin_df['confidence'].apply(ast.literal_eval)
cyrillic_df['confidence'] = cyrillic_df['confidence'].apply(ast.literal_eval)

Requires an anthropic key. I use a txt file that's ignored by the repo.

In [ ]:
import json
import anthropic
import os
import time
from itertools import islice
THRESHOLD = 0.3

if os.path.exists("OCR_lang_predictions.csv"):
    # If the file exists, load the dataset from the file
    df = pd.read_csv("OCR_lang_predictions.csv", index_col=0)
    
    # Find the last valid index where "lang_guess" is filled
    last_valid_idx = df["lang_guess"].last_valid_index()
    
    if last_valid_idx is not None:
        n = df.index.get_loc(last_valid_idx) + 1
        print(f"Resuming from index {n}")
    else:
        n = 0
        print("No valid 'lang_guess' values found. Starting from the beginning.")
else:
    n = 0
    print("File 'OCR_lang_predictions.csv' not found. Starting from the beginning.")


for idx, row in islice(latin_df.iterrows(), n, None):
    # Constructing the JSON prompt for each row.
    #print(row)
    if latin_df.at[idx, "flag"] == 1 or cyrillic_df.at[idx, "flag"] == 1:
        print(f"Flagged index {idx}")
        if latin_df.at[idx, "flag"] == 1:
            conf_ = latin_df.at[idx, "confidence"]
            text_ = latin_df.at[idx, "text"]
        else:
            conf_ = cyrillic_df.at[idx, "confidence"]
            text_ = cyrillic_df.at[idx, "text"]
        
        if latin_df.at[idx, "flag"] == 1 and cyrillic_df.at[idx, "flag"] == 1:
            if np.max(cyrillic_df.at[idx, 'confidence']) > np.max(latin_df.at[idx, 'confidence']):
                conf_ = cyrillic_df.at[idx, "confidence"]
                text_ = cyrillic_df.at[idx, "text"]

        # Sort text_ and conf_ based on the values in conf_
        #print(f"text_ is {text_}")
        sorted_indices = sorted(range(len(conf_)), key=lambda x: conf_[x], reverse=True)
        sorted_text = [text_[i] for i in sorted_indices]
        sorted_conf = [conf_[i] for i in sorted_indices]
        #print(f"sorted text is {sorted_text}")
        filtered_text = [text for text, conf in zip(sorted_text, sorted_conf) if conf >= THRESHOLD]
        filtered_conf = [conf for conf in sorted_conf if conf >= THRESHOLD]


        if len(filtered_text) == 0 :
        
            continue

        # Constructing JSON query.
        # Querying API
        print(f"Querying LLM for image {idx}.jpg, with peak confidence {np.max(filtered_conf)}, and filtered text {filtered_text}.")
        
        input_data = {
            "confidence": filtered_conf,
            "text": filtered_text
        }
        PROMPT = json.dumps(input_data, indent=2)

        parent_folder = os.path.dirname(os.path.dirname(__name__))

        claude_key = open(parent_folder + "claude_api_key.txt", "r").read()

        client = anthropic.Anthropic(
            api_key=claude_key,
            max_retries=2,  
            timeout=60.0    
        )

        try:
            message = client.messages.create(
                model="claude-3-opus-20240229",
                max_tokens=1000,
                temperature=0,
                system="You are tasked with determining the language of a string, or set of strings. The strings were extracted from randomly sampled images in a dataset. The strings may or may not be meaningless.\n\nThe input will be formatted as a JSON input. There will be two entries - \n1) A list of confidence metrics, ranging from 0 to 1, which is output from an EasyOCR text extraction model. One confidence metric for each string.\n2) A list of strings, which were extracted from the model.\n\nInput format is of the form:\n{\n  \"confidence\": [0.5, 0.3],\n  \"text\": [\"Extracted text 1\", \"Extracted text 2\"]\n}\n\nNaturally, the confidence at index i corresponds to the string at index i.\n\nPlease output two values:\n\n1) An integer confidence between 0 and 10. This does not need to be precise - I'd just like to differentiate between \"very confident\" and \"not at all confident\". As a model, you are effectively performing two functions: a) filtering out meaningless text OCR (for example, a response like \"Kf8xc\"), and b) identifying the language if and only if you're very confident.\n2) A language guess, using the 2-character standard identifier, e.g. \"en\" or \"de\"\n\n\nYour output should be of the form:\n\n{\n  \"confidence\": 3,\n  \"lang\": \"en\"\n}\n\nDo not include any additional information in your response - only return the JSON output as described above. \n\nIf multiple languages seem present (for instance, both Serbian (Latin) and Serbian (Cyrillic), I would like you to return both languages, sorted by the confidence, as lists. For instance:\n\n{\n  \"confidence\": [0.7, 0.4],\n  \"lang\": [\"en\", \"de\"]\n}\n\nIf a language has no matches (for instance, if the language is Mongolian or Chinese, which are not in the list of target languages), the confidence should be set to 0. \nIf all of the text is numeric (as would be expected from e.g. speed limit signs) the confidence should be low. \nIf the text seems like the metadata of a camera (for instance, data/time info from a dash-cam), confidence should be set very low, as the text is unrelated to text within the background. \nIf the text appears to have small typos (e.g. \"Rrestaurant\"), you may lower the confidence, but assume that this is due to variations in the font, or errors in the OCR model.\nIf the text is the name of a city, retail store, or other regional entity, (for instance, \"Wrocław\"), please return the dominant language spoken in that region (in this case, \"pl\").\n\n\nThe languages that you may choose from are as follows:\n\n{\n\"Belarusian\": \"be\",\n\"Bulgarian\": \"bg\",\n\"Czech\": \"cs\",\n\"Welsh\": \"cy\",\n\"Danish\": \"da\",\n\"German\": \"de\",\n\"English\": \"en\",\n\"Spanish\": \"es\",\n\"Estonian\": \"et\",\n\"French\": \"fr\",\n\"Irish\": \"ga\",\n\"Croatian\": \"hr\",\n\"Hungarian\": \"hu\",\n\"Icelandic\": \"is\",\n\"Italian\": \"it\",\n\"Latin\": \"la\",\n\"Lithuanian\": \"lt\",\n\"Latvian\": \"lv\",\n\"Maltese\": \"mt\",\n\"Dutch\": \"nl\",\n\"Norwegian\": \"no\",\n\"Polish\": \"pl\",\n\"Portuguese\": \"pt\",\n\"Romanian\": \"ro\",\n\"Russian\": \"ru\",\n\"Serbian (Latin)\": \"rs\",\n\"Serbian (Cyrillic)\": \"rc\",\n\"Slovak\": \"sk\",\n\"Slovenian\": \"sl\",\n\"Albanian\": \"sq\",\n\"Swedish\": \"sv\",\n\"Turkish\": \"tr\",\n\"Ukrainian\": \"uk\"\n}\n\nIf the input is invalid, please return instead error 4xx, and a description of why the input is invalid.\n\n{\n \"response\": 4xx,\n \"error\": \"The issue with the input\"\n}",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": PROMPT
                            }
                        ]
                    }
                ]
            )

        except anthropic.APIConnectionError as e:
            print(f"The server could not be reached for image {idx}.jpg")
            print(f"Underlying exception: {e.__cause__}")
            break
        except anthropic.RateLimitError as e:
            print(f"A 429 status code was received for image {idx}.jpg; backing off")
            break
            
        except anthropic.APIStatusError as e:
            print(f"A non-200 status code was received for image {idx}.jpg")
            print(f"Status code: {e.status_code}")
            print(f"Response: {e.response}")
            break
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response for image {idx}.jpg: {message.content}")
            break

        try:
            OUTPUT = json.loads(message.content[0].text)
        except:
            print(f"Some error with {idx}, giving output {message.content[0].text}. Skipping")
            break

        language_LLM = OUTPUT.get("lang")
        confidence_LLM = OUTPUT.get("confidence")

        if language_LLM is not None and confidence_LLM is not None:
            print(f"For image {idx}, detected language: {language_LLM}, LLM confidence score: {confidence_LLM}")
        else:
            print("Missing 'lang' or 'confidence' key in the JSON output.")
            
        # Including guess with confidence from LLM
        # Scaling from 1:10 to 0.1:1
        if isinstance(confidence_LLM, list):
            confidence_LLM = confidence_LLM[0]
            language_LLM = language_LLM[0]
            ### Flagging so I know there's multiple guesses
            df.at[idx, "flag"] = 1
        df.at[idx, "lang_guess"] = language_LLM
        df.at[idx, "conf"] = confidence_LLM 
        df.to_csv("OCR_lang_predictions.csv")


        time.sleep(30) # Waiting, so that I don't burn $500 instantly if this is more expensive than expected
        # I suppose it's not necessary since the model takes upwards of 10 seconds anyways. But, w/e.

In [ ]:
#### Generate a file containing the index and the OCR model confidence.
#### Pass this into a VLLM (Claude Opus or GPT-4 right now)
#### VLLM will output a language guess and a confidence score between 0:10.
#### Drop all guesses below threshold c.

import anthropic
import os
parent_folder = os.path.dirname(os.path.dirname(__name__))
#google_api_key = open(parent_folder + "google_maps_api_key.txt", "r").read()
claude_key = open(parent_folder + "claude_api_key.txt", "r").read()

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=claude_key
)
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    system="You are tasked with determining the language of a string, or set of strings. The strings were extracted from randomly sampled images in a dataset. The strings may or may not be meaningless.\n\nThe input will be formatted as a JSON input. There will be two entries - \n1) A list of confidence metrics, ranging from 0 to 1, which is output from an EasyOCR text extraction model. One confidence metric for each string.\n2) A list of strings, which were extracted from the model.\n\nInput format is of the form:\n{\n  \"confidence\": [0.5, 0.3],\n  \"text\": [\"Extracted text 1\", \"Extracted text 2\"]\n}\n\nNaturally, the confidence at index i corresponds to the string at index i.\n\nPlease output two values:\n\n1) An integer confidence between 0 and 10. This does not need to be precise - I'd just like to differentiate between \"very confident\" and \"not at all confident\". As a model, you are effectively performing two functions: a) filtering out meaningless text OCR (for example, a response like \"Kf8xc\"), and b) identifying the language if and only if you're very confident.\n2) A language guess, using the 2-character standard identifier, e.g. \"en\" or \"de\"\n\n\nYour output should be of the form:\n\n{\n  \"confidence\": 3,\n  \"lang\": \"en\"\n}\n\nDo not include any additional information in your response - only return the JSON output as described above. \n\nIf multiple languages seem present (for instance, both Serbian (Latin) and Serbian (Cyrillic), I would like you to return both languages, sorted by the confidence, as lists. For instance:\n\n{\n  \"confidence\": [0.7, 0.4],\n  \"lang\": [\"en\", \"de\"]\n}\n\nIf a language has no matches (for instance, if the language is Mongolian or Chinese, which are not in the list of target languages), the confidence should be set to 0. \nIf all of the text is numeric (as would be expected from e.g. speed limit signs) the confidence should be low. \nIf the text seems like the metadata of a camera (for instance, data/time info from a dash-cam), confidence should be set very low, as the text is unrelated to text within the background. \nIf the text appears to have small typos (e.g. \"Rrestaurant\"), you may lower the confidence, but assume that this is due to variations in the font, or errors in the OCR model.\nIf the text is the name of a city, retail store, or other regional entity, (for instance, \"Wrocław\"), please return the dominant language spoken in that region (in this case, \"pl\").\n\n\nThe languages that you may choose from are as follows:\n\n{\n\"Belarusian\": \"be\",\n\"Bulgarian\": \"bg\",\n\"Czech\": \"cs\",\n\"Welsh\": \"cy\",\n\"Danish\": \"da\",\n\"German\": \"de\",\n\"English\": \"en\",\n\"Spanish\": \"es\",\n\"Estonian\": \"et\",\n\"French\": \"fr\",\n\"Irish\": \"ga\",\n\"Croatian\": \"hr\",\n\"Hungarian\": \"hu\",\n\"Icelandic\": \"is\",\n\"Italian\": \"it\",\n\"Latin\": \"la\",\n\"Lithuanian\": \"lt\",\n\"Latvian\": \"lv\",\n\"Maltese\": \"mt\",\n\"Dutch\": \"nl\",\n\"Norwegian\": \"no\",\n\"Polish\": \"pl\",\n\"Portuguese\": \"pt\",\n\"Romanian\": \"ro\",\n\"Russian\": \"ru\",\n\"Serbian (Latin)\": \"rs\",\n\"Serbian (Cyrillic)\": \"rc\",\n\"Slovak\": \"sk\",\n\"Slovenian\": \"sl\",\n\"Albanian\": \"sq\",\n\"Swedish\": \"sv\",\n\"Turkish\": \"tr\",\n\"Ukrainian\": \"uk\"\n}\n\nIf the input is invalid, please return instead error 4xx, and a description of why the input is invalid.\n\n{\n \"response\": 4xx,\n \"error\": \"The issue with the input\"\n}",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "{\n  \"confidence\": [0.999],\n  \"text\": [\"MILGIAI\"]\n}\n\n"
                }
            ]
        }
    ]
)
print(message.content)

# Everything below is unused, since the Mapillary data does not have watermarks. 


### ~~Strip Google watermarks from detected text~~
### This is not necessary for the Mapillary dataset, but I'm leaving the algorithm in, since it may be useful to other devs.

Generating text for each image for each reader, then using manual pruning to determine which labels to drop for each language.
These can then be associated with the correct language.

I use the english reader, identify matching watermark text, determine the bounding box, and mask the image for that watermark.



In [ ]:
def OCR_extraction(path, reader):
    results =  reader.readtext(path)
    return results


In [ ]:

import os
os.makedirs("./Data/masked_for_ocr/", exist_ok=True)

reader_en = Readers["en"].reader

watermarks = ["gccole", "cgoogle", "cgccgle", "cgcogle", "cgocgle", "google", "gccgle", "gcogle", "gocgle", "gocnle", "gcogie", "oogle"]

### For each image, generate a mask (initially blank). Use this to mask results for the other readers.
for index, row in df.iterrows():
    
    ### Generating the mask with the english reader, filtering out the sections of the image containing the "Google" watermark.
    image = cv2.imread(row['img_path'])
    mask = np.ones(image.shape[:2], dtype=np.uint8) * 255
    results = OCR_extraction(row['img_path'], reader_en)
    
    bbox_ = []
    text = []
    confidence = []
    for result in results:
        bbox_.append(result[0])
        text.append(result[1])
        confidence.append(result[2])
    
    for idx, t in enumerate(text):
        ### Filtereing text matching the filter, or text with less than min_length characters (meaningless)
        min_length = 3
        if any(watermark.upper() in t.upper() for watermark in watermarks) or len(t) <= min_length:
            corners = bbox_[idx]
            x0, y0 = np.array(corners[0], dtype=np.int32) ### Top left corner
            x1, y1 = np.array(corners[2], dtype=np.int32) ### Bottom right corner
            cv2.rectangle(mask, (x0, y0), (x1, y1), 0, -1)
        else:
            print(f"Text {t.upper()} passed the filter for img index {index} and confidence {confidence[idx]}")
    
    ### We now have a masked image instead of the original image. Other lang readers can now use this image.
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    ### Store the images in a new directory (masked_for_ocr/) so that I only have to do this once
    cv2.imwrite(f"./Data/masked_for_ocr/{index}.png", masked_image)    
    

In [ ]:
masked_df = pd.read_csv('./Data/dataset/european_images.csv', index_col=0)
masked_df['img_path'] = [f"./Data/masked_for_ocr/{index}.png" for index in df.index]
masked_df['text'] =  pd.Series(dtype='object')
masked_df['confidence'] =  pd.Series(dtype='object')
masked_df['bbox'] =  pd.Series(dtype='object')

##### Now, with the text extracted from the masked images, we can set confidence thresholds, and only keep results above specific thresholds.

##### Identify the language with the highest confidence, and use this as our guessed label
##### Additionally, we can extract text and pass forward

Input the extracted text into an LLM. 